# GIN-forkに公開されているリポジトリから実験データを用意する

[GIN-fork](http://dg02.dg.rcos.nii.ac.jp/)に公開されているあなたのリポジトリや他の研究者のリポジトリのデータをこの実験の入力データもしくはソースコードとして用意します。  
指定したリポジトリにある実験のデータ（入力データ、ソースコード、出力データ）を、今回の実験で使えるように実験パッケージの`input_data`もしくは`source`配下にダウンロードします。  
上から順番にセルを実行してください。  
以下の図は、このタスクを行う際のユースケースです。ご参照ください。

<img src="./../../../files/WORKFLOWS/EX-WORKFLOWS/images/gin-forkユースケース.png" width=80%>

実験ワークフロー機能トップページに戻る場合は[こちら](../experiment.ipynb)。新規タブで開きます。  

## 0. 研究リポジトリ名と実験パッケージ名の確認  
以下のセルを実行すると、この実験実行環境で操作する実験パッケージの名前と、実験パッケージの存在する研究リポジトリ名を確認できます。  
研究リポジトリ名が表示されない場合は実験ワークフロー機能トップページのセクション「研究リポジトリ名・実験パッケージ名の確認」のセルを実行してください。  

In [ ]:
from IPython.display import display, HTML

# 研究リポジトリ名と実験パッケージ名を表示する
%store -r
if 'RESEARCH_TITLE' not in locals().keys() : RESEARCH_TITLE = '-'
if 'EXPERIMENT_TITLE' not in locals().keys() : EXPERIMENT_TITLE = '-'
res_text = "<h1 style='color: blue'>研究リポジトリ名：%s</h1>" % (RESEARCH_TITLE)
exp_text = "<h1 style='color: blue'>実験パッケージ名：%s</h1>" % (EXPERIMENT_TITLE)
display(HTML(res_text), HTML(exp_text))

## 1. 使いたいデータを持つリポジトリのURLを入力する
以下のセルを実行して、今回の実験で用いたいデータのある研究リポジトリのGIT URLを入力してください。  
GIT URLについては、以下の図を参考に、GIN-forkの研究リポジトリの画面右上でコピーしてください。  
<img src="./../../../files/WORKFLOWS/EX-WORKFLOWS/images/gitURL.png" width=60%>  
入力したURLの履歴をこのノートブックに保存したい場合は、実行後にセルの左側に表示されるピンマークをクリックしてください。  
※入力に誤りがある場合は、再度このセル以降を実行して下さい。

In [ ]:
from ipywidgets import Text, Button, Layout
from IPython.display import clear_output

def on_click_callback(clicked_button: Button) -> None:
    global input_repo
    global input_repo_title
    global input_repo_path
    input_repo = text.value
    input_repo_title = input_repo.split('/')[-1].replace('.git', '')
    input_repo_path = '/home/jovyan/.tmp/' + input_repo_title
    clear_output()
    print("入力完了しました：", input_repo)

# テキストボックス
text = Text(
    description='URL：',
    placeholder='http://dg02.dg.rcos.nii.ac.jp/user/repository_title.git',
    layout=Layout(width='500px')
)
button = Button(description='入力完了')
button.on_click(on_click_callback)
text.on_submit(on_click_callback)
display(text, button)

In [ ]:
# 入力されたリポジトリについてGUIを表示するため軽量にクローンする
import os
import traceback
from IPython.display import HTML, display, clear_output
!mkdir -p .tmp
try:
    !git clone -b master --depth 1 --filter=blob:none $input_repo /home/jovyan/.tmp/$input_repo_title
    os.chdir('/home/jovyan/.tmp/' + input_repo_title)
    # git annex whereisからURLを取得するために必要
    !git fetch origin git-annex:remotes/origin/git-annex
    os.chdir(os.environ['HOME'])
except Exception:
    display(HTML("<p><font color='red'>入力されたリポジトリに接続できません。公開リポジトリとして存在しているか、また入力したURLに間違いがないかを確認してください。</font></p>"))
    print(traceback.format_exc())
else:
    clear_output()
    display(HTML("<p>入力されたリポジトリに接続できました。次の処理にお進みください。</p>"))

## 2. 必要なデータを持つ実験名を選択する

以下のセルを実行して、1.で入力したリポジトリの実験群から必要なデータを持つ実験名を選択してください。  
パラメータ実験の場合は、パラメータ実験名も選択してください。  
同じリポジトリの複数の実験からファイルを用意したい場合は、各実験毎に手順2.～6.を繰り返し実行してください。

In [ ]:
import json
import glob 
from ipywidgets import Dropdown, Button, Layout
from IPython.display import clear_output


# 入力リポジトリの実験群を取得
input_repo_path = '/home/jovyan/.tmp/' + input_repo.split('/')[-1].replace('.git', '')
with open(input_repo_path + '/pipeline.json', 'r') as f:
    pipeline = json.load(f)

# 選択画面生成
style = {'description_width': 'initial'}
print('取得したいデータを持つ実験パッケージ名を以下から選択し、入力完了ボタンをクリック下さい。')
    
def on_click_package(clicked_button: Button) -> None:
    def on_click_parameter(clicked_button: Button) -> None:
        global parameter
        print("HERE")
        parameter=dropdown_parameter.value
        clear_output()
        print('入力を受けつけました。')
        print('実験パッケージ名：', package)
        print('パラメータ実験名：', parameter)
    
    global package
    package=dropdown_package.value
    clear_output()
    print('入力を受けつけました。')
    print('実験パッケージ名：', package)
    
    # for_parametersの場合は、パラメータフォルダも選択
    with open(input_repo_path + '/dmp.json', 'r') as f:
        global dmp
        dmp = json.load(f)
    if dmp['datasetStructure'] == 'for_parameters':
        # パラメータフォルダ一覧を生成
        output_dirs = glob.glob(input_repo_path + '/experiments/' + package + '/**/output_data/', recursive=True)
        parameter_dirs = [dir.replace('/output_data/', '') for dir in output_dirs]
        parameter_dirs = [dir.replace(input_repo_path + '/experiments/' + package + '/', '') for dir in parameter_dirs]
        clear_output()
        print('取得したい出力データを持つパラメータ実験名を選択し、入力完了ボタンをクリック下さい。')
        
        dropdown_parameter = Dropdown(
            options=parameter_dirs,
            description='パラメータ実験名:',
            disabled=False,
            layout=Layout(width='initial'),
            style=style
        )
        button_parameter = Button(description='入力完了する')
        button_parameter.on_click(on_click_parameter)
        display(dropdown_parameter, button_parameter)
    else:
        pass

dropdown_package = Dropdown(
    options=pipeline,
    description='実験パッケージ名:',
    disabled=False,
    layout=Layout(width='initial'),
    style=style
)

button_package = Button(description='入力完了')
button_package.on_click(on_click_package)
display(dropdown_package, button_package)

## 3. 必要なデータを選択する

以下のセルを実行すると、手順2.で選択した実験にあるデータが表示されます。  
表示されたデータからこの実験に必要なデータをクリックして選択してください。  
Ctrlキーを押しながらクリックすると複数のデータを選択できます。

In [ ]:
import glob
import panel as pn
from pathlib import Path

pn.extension()
package_path = input_repo_path + '/experiments/' + package

# 選択完了ボタン押下後
def gen_gui_list(event):
    done_button.button_type = "success"
    done_button.name = "選択完了しました。次の処理にお進みください。"
    global gui_list
    gui_list = []
    for i in range(len(column)):
        if len(column[i].value) > 0:
            gui_list.append('### ' + column[i].name)
        for index in range(len(column[i].value)):
            gui_list.append(pn.widgets.TextInput(name=column[i].name + '/' + column[i].value[index], placeholder='Enter a file path here...', width=700))

    
done_button = pn.widgets.Button(name= "選択を完了する", button_type= "primary")
done_button.on_click(gen_gui_list)

# input_data, source, output_data毎にファイルの一覧を作成する
def get_files(target, parameter):
    if parameter == '':
        cmd_glob = package_path + '/' + target + '/**'
        cmd_replace = package_path + '/' + target + '/'
    else:
        cmd_glob = package_path + '/' + parameter + '/' + target + '/**'
        cmd_replace = package_path + '/' + parameter + '/' + target + '/'
    files = glob.glob(cmd_glob, recursive=True)
    for i in files[:]:
        p = Path(i)
        if Path.is_dir(p):
            files.remove(i)
    return [file.replace(cmd_replace, '') for file in files]

# input_data, source, output_data毎のMultiSelect画面を作成し、リストを返す
def generate_gui(files_list):
    global gui_list
    gui_list = []
    for key, value in files_list.items():
        if key == 'input_data' or  key == 'source' or key =='output_data':
            gui_list.append(pn.widgets.MultiSelect(name=key, options=value, size=8, sizing_mode='stretch_width'))
        elif key == 'parameter_output_data':
            gui_list.append(pn.widgets.MultiSelect(name=parameter + '/' + key.replace('parameter_',''), options=value, size=8, sizing_mode='stretch_width'))
    return gui_list
            
# 実験パッケージのタイプごとにGUIを生成する
if dmp['datasetStructure'] == 'with_code':
    input_data_files = get_files(target='input_data', parameter='')
    source_files = get_files(target='source', parameter='')
    output_data_files = get_files(target='output_data', parameter='')
    files_list = {"input_data":input_data_files, "source":source_files, "output_data":output_data_files}
    gui = generate_gui(files_list)
    
elif dmp['datasetStructure'] == 'for_parameters':
    input_data_files = get_files(target='input_data', parameter='')
    source_files = get_files(target='source', parameter='')
    output_data_files = get_files(target='output_data', parameter=parameter)
    files_list = {"input_data":input_data_files, "source":source_files, "parameter_output_data":output_data_files}
    gui = generate_gui(files_list)

# GUIを表示する
column = pn.Column()
for target in gui:
    column.append(target)
column.append(done_button)
column

## 4. 選択したデータの格納先を入力する

手順3で選択したデータの格納先について、実験パッケージからファイル名までのパスを入力してください。  
例えば、`input_data/experiment_1_output/sample.txt`などです。  
<p style="color:red;">格納先のファイルパスは、`input_data/`, `source/`で始まる必要があります。</p> 

In [ ]:
import panel as pn
from IPython.display import HTML, display

# 選択完了ボタン押下後
def verify_input_text(event):
    for i in range(len(column)):
        panel_type = str(type(column[i]))
        if 'TextInput' in panel_type:
            if column[i].value.startswith('input_data/') or column[i].name.startswith('source/'):
                done_button.button_type = "success"
                done_button.name = "入力を完了しました。次の処理にお進みください。"
            else:
                done_button.button_type = "danger"
                done_button.name = "`input_data/`か`source/`で始まる必要があります。修正後、再度クリックしてください。"
                break
            
done_button = pn.widgets.Button(name= "入力を完了する", button_type= "primary")
done_button.on_click(verify_input_text)
column = pn.Column()
for target in gui_list:
    column.append(target)
column.append(done_button)
column

## 5. 選択したデータのリンクを指定した格納先に用意する

In [ ]:
import os
import csv
import traceback
from datalad import api
from IPython.display import HTML, display, clear_output

# datalad addurlsで使うファイルのダウンロードリンクと格納先のcsvファイルを生成する
dest = {}
%store -r EXPERIMENT_TITLE
for i in range(len(column)):
    panel_type = str(type(column[i]))
    if 'TextInput' in panel_type:
        cmd = package_path + '/' + column[i].name
        annex_key = !git annex info $cmd | grep key
        if 'URL' in annex_key[0].split('-')[0]:
            # S3などインターネット上に実体がある場合
            os.chdir('/home/jovyan/.tmp/' + input_repo_title)
            annex_key = annex_key[0].replace('key: ', '')
            key = !git annex info $cmd | grep 'key: '
            key = key[0].replace("key: ", "")
            key = key.replace("'", "")
            results = !git annex whereis --key $key
            url = ''
            for r in results:
                if r.startswith('  web: '):
                    url = r.replace('  web: ', '')
            dest_path = 'experiments/' + EXPERIMENT_TITLE + '/' + column[i].value
            dest[url] = dest_path
        else:
            # 実体がGIN-forkにある場合
            url = input_repo.replace('.git', '') + '/raw/master/experiments/' + package + '/' + column[i].name
            dest_path = 'experiments/' + EXPERIMENT_TITLE + '/' + column[i].value
            dest[url] = dest_path
with open('/home/jovyan/.tmp/datalad-addurls.csv', 'w+') as f:
    writer = csv.writer(f)
    writer.writerow(['who','link'])
    for key, value in dest.items():
        writer.writerow([value, key])

# addurlsを実行してgit annexリンクを作成する
os.chdir(os.environ['HOME'])
result = ''
try:
    result = !datalad addurls --nosave --fast .tmp/datalad-addurls.csv '{link}' '{who}'
    for line in result:
        if 'addurls(error)' in line  or 'addurls(impossible)' in line:
            raise Exception
except Exception:
    display(HTML("<p><font color='red'>リンクの作成に失敗しました。用意したいデータにアクセス可能か確認してください。</font></p>"))
    print(traceback.format_exc())
else:
    clear_output()
    display(HTML("<p>リンクの作成に成功しました。次の処理にお進みください。</p>"))


## 6. データの来歴の記録と、実データのダウンロードを行う

In [ ]:
import os
import traceback
from datalad import api
from IPython.display import HTML, display, clear_output

try:
    # このタスクで用意したファイルパス
    datalad_get_path = []
    for value in dest.values():
        datalad_get_path.append(value)

    # sourceの来歴を残すため一旦annex管理でコミットする
    os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
    from util.scripts import utils
    os.chdir(os.environ['HOME'])
    utils.syncs_with_repo(git_path=None, gitannex_path=datalad_get_path, message=input_repo_title + 'リポジトリの' + package + '実験から実験のデータを用意')

    # 実データを取得する
    api.get(path=datalad_get_path)

    # sourceがあれば、git管理にする
    source_path = []
    for path in datalad_get_path:
        if path.startswith('experiments/' + EXPERIMENT_TITLE + '/source'):
            source_path.append(path)
    if len(source_path) > 0:
        for path in source_path:
            !git annex unannex $path
except Exception:
    display(HTML("<p><font color='red'>処理に失敗しました。用意したいデータにアクセス可能か確認してください。</font></p>"))
    print(traceback.format_exc())
else:
    clear_output()
    display(HTML("<p>来歴の記録とデータのダウンロードに成功しました。次の処理にお進みください。</p>"))


## 7. 本タスクの実行結果をGIN-forkに同期する

ここまでの内容を保存し、GIN-forkに同期します。  
以下のセルを実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[こちらのFAQ](http://dg02.dg.rcos.nii.ac.jp/G-Node/Info/wiki/%E3%83%AF%E3%83%BC%E3%82%AF%E3%83%95%E3%83%AD%E3%83%BC#1-1%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%E3%81%A8%E3%80%81%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E5%81%B4%E3%81%AE%E5%A4%89%E6%9B%B4%E3%81%A8%E7%AB%B6%E5%90%88%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82%E7%AB%B6%E5%90%88%E3%82%92%E8%A7%A3%E6%B1%BA%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82%E3%81%A8%E8%A1%A8%E7%A4%BA%E3%81%95%E3%82%8C%E3%82%8B)を参考に競合を解決してください。

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
os.chdir(os.environ['HOME'])

git_path = source_path
git_path.append('WORKFLOWS/EX-WORKFLOWS/prepare_from_repository.ipynb')

utils.syncs_with_repo(git_path=git_path, gitannex_path=None, message=EXPERIMENT_TITLE + '_実験データの用意')

## 8. 不要なデータを削除する



In [ ]:
# 一時的なクローンを削除する
import os
os.chdir(os.environ['HOME'])
!rm -rf .tmp/$input_repo_title
print("処理が完了しました。")

## 9. 実験ワークフロー機能トップページに遷移する

以下のセルを実行し、表示されるリンクをクリックして実験ワークフロー機能トップページに戻ってください。  

In [ ]:
from IPython.display import display, HTML, Javascript
display(HTML("<a href='../experiment.ipynb'>実験ワークフロー機能トップページに遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))